# Mapreader Workshops 2024

----

First check you have the correct version of MapReader: v1.3.0

This can be downloaded from pypi using `pip install mapreader==1.3.0` or by checking out the repo at [this commit](https://github.com/Living-with-machines/MapReader/releases/tag/v1.3.0)

In [1]:
import mapreader
assert mapreader.__version__ == '1.3.0'

-------------

# Download

https://mapreader.readthedocs.io/en/latest/User-guide/Download.html

In this notebook, we will download pre-selected six maps for use in the workshop. 

Again, we will use the ``SheetDownloader`` and metadata from the NLS stored in the ``persistent_data`` directory of the mapreader repository.
This can be set up in the same way as the previous notebook.

## Set up the ``SheetDownloader``

In [ ]:
from mapreader import SheetDownloader

To get your download URL, please set up a free account on [maptiler](https://cloud.maptiler.com/maps/) and go to https://cloud.maptiler.com/tiles/uk-osgb10k1888/ to find the tile layer for the 2nd edition OS 6inch maps of the UK.

The URL you will need is the XYZ url listed under the Raster tiles heading.

__**YOUR TURN**__: Set up your ``my_ts`` by passing the ``metadata_path`` and ``download_url`` arguments to the ``SheetDownloader`` .

See [here](https://mapreader.readthedocs.io/en/latest/User-guide/Download.html#sheetdownloader) in the docs.

In [ ]:
# my_ts = SheetDownloader(
#    metadata_path="../NLS_metadata/metadata_OS_Six_Inch_GB_WFS_light.json",
#    download_url="",
# )

## Select maps to download

In [ ]:
# This is a preselected list of map sheets from the coast near Glasgow in Scotland
nls_image_url_ids = [
    "75650661",
    "75650697",
    "75650823",
    "75650907",
    "75650892",
    "75650931",
]

In [ ]:
my_ts.found_queries = []

In [ ]:
for nls_image_url_id in nls_image_url_ids:
    my_ts.query_map_sheets_by_string(
        nls_image_url_id, keys=["properties", "IMAGEURL"], append=True
    )

In [ ]:
len(my_ts.found_queries)

In [ ]:
my_ts.extract_wfs_id_nos()
my_ts.plot_queries_on_map(map_extent="uk", add_id=False)

## Download query results

To download the image files of the maps from the NLS tile server, you need to first set a zoom level using the ``.get_grid_bb()`` method.

We are using zoom level 17 - which will give us high resolution images.

In [ ]:
my_ts.get_grid_bb(17)

Then, you can download your map sheets using ``.download_map_sheets_by_queries()``:

> _**NOTE**_: We have left ``path_save`` and ``metadata_fname`` as the default values, so maps will be saved in ``"./maps/"`` and their metadata will be saved as ``"./maps/metadata.csv"``.

In [ ]:
my_ts.download_map_sheets_by_queries()

-----

# Load

MapReader’s ``Load`` subpackage is used to load, visualize and patchify images (e.g. maps) saved locally.

In this workshop, we will load the images that we have just downloaded from the NLS tile server.
These are saved in ``"./maps/"``.

However, you could use any images that you have saved locally. 
See the [Input Guidance](https://mapreader.readthedocs.io/en/latest/Input-guidance.html) section of the MapReader documentation for more details about file formats and metadata requirements.

## Import the ``loader`` and create your ``my_files`` object.

In [ ]:
from mapreader import loader

__**YOUR TURN**__: Load your maps. They are saved in ``"./maps/"`` as ``"*.png"`` files.

See [here](https://mapreader.readthedocs.io/en/latest/User-guide/Load.html#load-images-and-metadata) in the docs.

In [ ]:
# my_files = loader()

### Add metadata

Add the ``metadata.csv`` file that was created when downloading your maps.

__**YOUR TURN**__: Add your metadata. It is saved in ``"./maps/metadata.csv"``

See [here](https://mapreader.readthedocs.io/en/latest/User-guide/Load.html#load-images-and-metadata) in the docs.

In [ ]:
# my_files.add_metadata()

In [ ]:
parent_df, patch_df = my_files.convert_images()
parent_df.head()

## Patchify

The ``.patchify_all()`` method is used to slice your map images into patches. 

The method used to patchify your maps is selected by specifying ``method="pixel"`` or ``method="meters"``. This determines whether your ``patch_size`` is interpreted with units of ``pixel`` or ``meters``. 

#### Patchify by pixel

> _**NOTE**_: We have used ``add_to_parents=False`` here so that these patches are not added to the ``my_files`` object. This is simply so that we don't have two sets of patches added after running both ``.patchify_all()`` commands.

In [ ]:
my_files.patchify_all(method="pixel", patch_size=1000, add_to_parents=False)

#### Patchify by meters

You will only be able to use ``method="meters"`` if you have coordinates saved for each of your map images. 
These coordinates should correspond to the "bounding box" of your map image (minimum x, minimum y, maximum x and maximum y) and thereby associate the left, bottom, right and top edges of your map image to their geospatial locations.

These can be added by running either ``.add_metadata()`` (assuming your metadata contains these coordinates) or, if your images contain georefencing information, ``.add_geo_info()``.

__**YOUR TURN**__: Patchify your maps into 100 meter patches. Unlike last time, we do want to load these patches into our `my_files` object and so we should set ``add_to_parents=True``.

See [here](https://mapreader.readthedocs.io/en/latest/User-guide/Load.html#patchify) in the docs.

In [ ]:
# my_files.patchify_all(method="", patch_size=)

You will see your patches are saved in separate directories, each indicating the patch size and method used.

### Visualize results

``MapReader`` also contains some useful functions for visualizing your patches.

For example, the ``.show_sample()`` method can be used to show a random sample of your patches:

In [ ]:
my_files.show_sample(num_samples=3, tree_level="patch", random_seed=1)

You may also want to see all the patches created from one of your parent images. 
This can be done using the ``.show_parent()`` method:

In [ ]:
my_files.show_parent("map_75650661.png")

The ``.calc_pixel_stats()`` method can be used to calculate statistics on the pixel intensities of each patch.

Patches with higher pixel intensities are closer to white and so are more likely to contain blank space whereas patches with lower pixel intensities are more likely to contain features. Knowing pixel intensities can be useful for sorting patches when annotating. 

In [ ]:
my_files.calc_pixel_stats()

By running the ``.convert_images()`` method, you will see that the means and standard deviations of pixel intensities of each patch have been added to your ``my_files`` object.

In [ ]:
parent_df, patch_df = my_files.convert_images()
patch_df.head()

To save these outputs, use the ``save=True`` argument.

> _**NOTE**_: By default, this will save your outputs as ``.csv`` files. If instead, you'd like to save as ``.xslx`` files, add ``save_format="excel"`` to your command.

In [ ]:
parent_df, patch_df = my_files.convert_images(save=True)

In [ ]:
parent_df, patch_df = my_files.convert_images(save=True, save_format="excel")

Or to save your outputs as a `.geojson` file which can be loaded into GIS software (e.g QGIS), you can use the `save_patches_to_geojson()` method.

For now, this will just contain all the information in your patch dataframe but will eventually contain all the patch predictions and confidence scores too.

In [ ]:
my_files.save_patches_to_geojson("test_patches.geojson")

-----

# Documentation

Please refer to the [MapReader documentation](https://mapreader.readthedocs.io/en/latest/) for more information.

---------

Now head to part 2 of the workshop notebooks!